In [1]:
import pandas as pd
import numpy as np


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge

In [3]:
data = pd.read_csv('data/salary-train.csv')
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [4]:
test = pd.read_csv('data/salary-test-mini.csv')
test['FullDescription'] = test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
for i, text in enumerate(test.FullDescription):
    test.FullDescription[i] = text.lower()

c:\users\nout\.conda\envs\jpytest\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
data['FullDescription'] = data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [6]:
for i, text in enumerate(data.FullDescription):
    data.FullDescription[i] = text.lower()

c:\users\nout\.conda\envs\jpytest\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
data

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355
...,...,...,...,...
59995,as a result of continued growth first class s...,Whitley Bay,contract,26400
59996,php mvc web developer macclesfieldcirca ...,Macclesfield,permanent,26000
59997,staff nurse nursing home baldock white recru...,Baldock,NaN,24500
59998,this is one of the best agency side opportunit...,The City,permanent,65000


In [16]:
vectorizer = TfidfVectorizer(min_df=5)

X_train_vec = vectorizer.fit_transform(data['FullDescription'])

X_test_vec = vectorizer.transform(test['FullDescription'])

In [23]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [22]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

test['LocationNormalized'].fillna('nan', inplace=True)
test['ContractTime'].fillna('nan', inplace=True)


In [13]:
data

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355
...,...,...,...,...
59995,as a result of continued growth first class s...,Whitley Bay,contract,26400
59996,php mvc web developer macclesfieldcirca ...,Macclesfield,permanent,26000
59997,staff nurse nursing home baldock white recru...,Baldock,nan,24500
59998,this is one of the best agency side opportunit...,The City,permanent,65000


In [14]:
X_train_categ

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [11]:
import scipy.sparse as sparse 

In [24]:
X_for_train = sparse.hstack([X_train_vec, X_train_categ])

X_for_test = sparse.hstack([X_test_vec, X_test_categ])

In [25]:
model = Ridge(alpha=1, random_state=241)

model.fit(X_for_train, data.SalaryNormalized)

model.predict(X_for_test)

array([56565.3254579 , 37140.63063337])